# Init SparkContext

In [1]:
from datetime import datetime
from pyspark import SparkContext, HiveContext
from pyspark.sql import SparkSession, SQLContext

In [2]:
spark = (SparkSession.builder.appName("pyspark-dataframe-demo-{}".format(datetime.today()))
        .master("spark://spark-master:7077")      
        .getOrCreate())

sqlContext = SQLContext(spark)
spark.sparkContext.getConf().getAll()


/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[('spark.repl.local.jars',
  'file:///usr/local/spark-3.3.1-bin-hadoop3/jars/delta-core_2.12-2.2.0.jar,file:///usr/local/spark-3.3.1-bin-hadoop3/jars/hadoop-aws-3.3.2.jar,file:///usr/local/spark-3.3.1-bin-hadoop3/jars/delta-storage-2.2.0.jar,file:///usr/local/spark-3.3.1-bin-hadoop3/jars/aws-java-sdk-1.12.367.jar,file:///usr/local/spark-3.3.1-bin-hadoop3/jars/s3-2.18.41.jar,file:///usr/local/spark-3.3.1-bin-hadoop3/jars/aws-java-sdk-bundle-1.11.1026.jar'),
 ('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false'),
 ('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.conc

# Create DataFrame

## By loading dataset

In [3]:
df_orders = spark.read.format("csv").load("s3a://warehouse/olist_orders_dataset.csv", header=True)
df_orders.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [5]:
(
df_orders.write.mode("overwrite")
    .option("compression", "snappy")
    .option("path", "s3a://warehouse/olist_orders_dataset.parquet")
    .format("parquet")
    .saveAsTable("olist_orders_dataset")
)

# Spark SQL

In [6]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [7]:
spark.sql("show tables").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|olist_orders_dataset|      false|
+---------+--------------------+-----------+



In [8]:
spark.sql("SELECT order_id, customer_id, order_status, order_purchase_timestamp FROM olist_orders_dataset").show()

+--------------------+--------------------+------------+------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|
+--------------------+--------------------+------------+------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|   delivered|     2018-08-08 08:38:49|
|949d5b44dbf5de918...|f88197465ea7920ad...|   delivered|     2017-11-18 19:28:06|
|ad21c59c0840e6cb8...|8ab97904e6daea886...|   delivered|     2018-02-13 21:18:39|
|a4591c265e18cb1dc...|503740e9ca751ccdd...|   delivered|     2017-07-09 21:57:05|
|136cce7faa42fdb2c...|ed0271e0b7da060a3...|    invoiced|     2017-04-11 12:22:08|
|6514b8ad8028c9f2c...|9bdf08b4b3b52b552...|   delivered|     2017-05-16 13:10:30|
|76c6e866289321a7c...|f54a9f0e6b351c431...|   delivered|     2017-01-23 18:29:09|
|e69bfb5eb88e0ed

In [10]:
spark.sql("SELECT COUNT(*) AS number_of_records FROM olist_orders_dataset").show()

+-----------------+
|number_of_records|
+-----------------+
|            99441|
+-----------------+

